TimeSeries
==========

The [TimeSeries](api/kineticstoolkit.TimeSeries.rst) class is the basis of most Kinetic Toolkit's modules and is the only class that users have to learn after python standard types and numpy arrays.

------------------------

***The three roles of the TimeSeries class are to:***

1. *organize multidimensional data in time;*
2. *deal with events;*
3. *associate metadata to data.*

------------------------

TimeSeries are largely inspired by Matlab's `timeseries` and `tscollection`.


TimeSeries basics
-----------------

Every TimeSeries contains the following attributes:

- `time`: A numpy array that contains the time vector.
- `data`: A dict where each entry is a numpy array, with the first dimension corresponding to time.
- `events`: An optional list of events.
- `time_info`: Metadata corresponding to time, that contains at least the time unit.
- `data_info`: Optional metadata.

A TimeSeries in its simplest form contains a time vector and at least one data series. For example:

In [ ]:
import kineticstoolkit.lab as ktk
import numpy as np

ts = ktk.TimeSeries()
ts.time = np.arange(0, 10, 0.1)  # 10 seconds at 10 Hz
ts.data['Sinus'] = np.sin(ts.time)

ts

In [ ]:
ts.data

TimeSeries can be [plotted](api/kineticstoolkit.TimeSeries.plot.rst) directly using Matplotlib:

In [ ]:
ts.plot()

A TimeSeries can contain many independent data that share a same time vector:

In [ ]:
ts.data['Cosinus'] = np.cos(ts.time)

ts

In [ ]:
ts.data

In [ ]:
ts.plot()

A TimeSeries can also contain multidimensional data, as long as the first dimension corresponds to time. Here, we simulate a moving kinematics marker.

In [ ]:
# Initialize the numpy array to 100 samples of a (x, y, z, 1) position vector.
ts.data['Marker1'] = np.empty((100, 4))

# Simulate a randomly moving marker
np.random.seed(0)
ts.data['Marker1'][:, 0] = np.cumsum(np.cumsum(0.01 * (np.random.rand(100) - 0.5)))  # x
ts.data['Marker1'][:, 1] = np.cumsum(np.cumsum(0.01 * (np.random.rand(100) - 0.5)))  # y
ts.data['Marker1'][:, 2] = np.cumsum(np.cumsum(0.01 * (np.random.rand(100) - 0.5)))  # z
ts.data['Marker1'][:, 3] = 1                                                         # 1

ts.data

In [ ]:
ts.plot()

Exporting and importing pandas DataFrames
-----------------------------------------

TimeSeries integrate well with pandas DataFrames and therefore with a plethora or data analysis softwares, using the [TimeSeries.to_dataframe()](api/kineticstoolkit.TimeSeries.to_dataframe.rst) and [TimeSeries.from_dataframe()](api/kineticstoolkit.TimeSeries.from_dataframe.rst) methods. For example, exporting the previous TimeSeries to a DataFrame gives:

In [ ]:
df = ts.to_dataframe()

df

Note the brackets in the Marker1 headers that indicate multidimensional data. For higher dimensions, these brackets would multiple indexes: for example, a series of rigid 4x4 transformation matrices would require 16 columns and the indexes would go from [0,0] to [3,3].

Now, importing from a DataFrame:

In [ ]:
ts2 = ktk.TimeSeries.from_dataframe(df)

ts2.plot()

For the rest of this tutorial, we will work with wheelchair propulsion kinetic data from a CSV file.

In [ ]:
import pandas as pd

# Read some columns
df = pd.read_csv(ktk.config.root_folder + '/data/timeseries/smartwheel.csv',
                 usecols=[18, 19, 20, 21, 22, 23],
                 names=['Forces[0]', 'Forces[1]', 'Forces[2]',
                        'Moments[0]', 'Moments[1]', 'Moments[2]'],
                 nrows=5000)

# Assign time to the DataFrame's index, where the sampling rate is 240 Hz.
df.index = np.arange(df.shape[0]) / 240
df

Now, we convert this DataFrame to a TimeSeries:

In [ ]:
ts = ktk.TimeSeries.from_dataframe(df)

ts

In [ ]:
ts.data

In [ ]:
ts.plot()

Metadata
--------

The `time_info` property associates metadata to the time vector. It is a dictionary where each key is the name of one metadata. By default, `time_info` includes the `Unit` metadata, which corresponds to `s`. Any other metadata can be added by adding new keys in `time_info`.

In [ ]:
ts.time_info

Similarly, the `data_info` property associates metadata to data. This property is a dictionary of dictionaries, where the outer key corresponds to the data key, and the inner key is the metadata. The [TimeSeries.add_data_info()](api/kineticstoolkit.TimeSeries.add_data_info.rst) method eases the management of `data_info`.

In [ ]:
ts = ts.add_data_info('Forces', 'Unit', 'N')
ts = ts.add_data_info('Moments', 'Unit', 'Nm')

Unless explicitly mentioned, metadata is not used for calculation and is optional. It is simply a way to clarify the data by adding information to it. Some functions however read metadata: for example, the [TimeSeries.plot()](api/kineticstoolkit.TimeSeries.plot.rst) method looks for possible `Unit` metadata and prints it on the y axis.

In [ ]:
ts.plot()

Events
------

In the figure above, we see that the TimeSeries contains cyclic data that could be characterized by events. A first spike was generated at about 4 seconds: this event corresponds to a synchronization signal that we generated by gently impacting the instrumented pushrim. Thereafter, we see a series of pushes and recoveries.

We will add these events to the TimeSeries by hand using [TimeSeries.add_event()](api/kineticstoolkit.TimeSeries.add_event.rst). It is also possible to add events interactively using [TimeSeries.ui_add_event()](api/kineticstoolkit.TimeSeries.ui_add_event.rst). Please see the tutorial on the [cycles](cycles.rst) module to learn how to detect cycles automatically.

In [ ]:
ts = ts.add_event(4.35, 'sync')
ts = ts.add_event(8.56, 'push')
ts = ts.add_event(9.93, 'recovery')
ts = ts.add_event(10.50, 'push')
ts = ts.add_event(11.12, 'recovery')
ts = ts.add_event(11.78, 'push')
ts = ts.add_event(12.33, 'recovery')
ts = ts.add_event(13.39, 'push')
ts = ts.add_event(13.88, 'recovery')
ts = ts.add_event(14.86, 'push')
ts = ts.add_event(15.30, 'recovery')

These events are now added to the TimeSeries' list of events:

In [ ]:
ts

In [ ]:
ts.events

If we plot again the TimeSeries, we can see the added events.

In [ ]:
ts.plot()

### Using events to synchronize TimeSeries ###

Let's see how we can make use of these events. First, the `sync` event can be used to set the zero-time. This would be useful to sync this TimeSeries with data from another wheel, or with any another instrument that also has such synchronization event. The [TimeSeries.sync_event()](api/kineticstoolkit.TimeSeries.sync_event.rst) shifts the TimeSeries' time and every event's time so that the sync event becomes the new "zero-time".

In [ ]:
ts = ts.sync_event('sync')
ts.plot()

### Using events to extract shorter TimeSeries ###

The `TimeSeries` class comes with a myriad of methods such as [TimeSeries.get_ts_after_event()](api/kineticstoolkit.TimeSeries.get_ts_after_event.rst), [TimeSeries.get_ts_between_events()](api/kineticstoolkit.TimeSeries.get_ts_between_events.rst), etc. For example, if we want to analyze data of the four first pushes and get rid of any other data, we could extract a new TimeSeries that contains only these data:

In [ ]:
# Extract data push event 0 up to push event 4.
ts2 = ts.get_ts_between_events('push', 'push', 0, 4, inclusive=True)

# Remove the events that are not contained into the new time range.
ts2 = ts2.trim_events()

ts2.plot()

Subsetting and merging timeseries
---------------------------------

We can use the [TimeSeries.get_subset()](api/kineticstoolkit.TimeSeries.get_subset.rst) method to extract some signals from a TimeSeries. For example, if we only want to keep force information and get rid of the moments:

In [ ]:
ts3 = ts2.get_subset(['Forces'])

ts3.data

In [ ]:
ts3.plot()

For more information, please refer to the [API reference for the TimeSeries class](api/kineticstoolkit.timeseries.rst).